# Módulo 10: Llamado de Variantes

## Descripción general

En esta sección, aprenderemos a mapear lecturas de secuencias a una secuencia de referencia y, a continuación, a identificar las variantes (tanto polimorfismos de un solo nucleótido (SNP) como pequeñas inserciones y deleciones (indels)) en los datos de la secuencia. A continuación, utilizaremos las variantes identificadas para generar un pseudogenoma que se crea sustituyendo las bases del genoma de referencia por las variantes identificadas en las posiciones correspondientes.

### Instalar condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Instalar programas

In [ ]:
# Instalar bwa
!conda install -c conda-forge -c bioconda -c defaults snippy

In [ ]:
# Install samtools
!conda config --add channels bioconda
!conda config --add channels conda-forge
!conda install -c bioconda samtools

### Descargar datos

In [ ]:
!wget

## Parte 1: Alineamiento

Utilizaremos la herramienta [Snippy](https://github.com/tseemann/snippy), que es un programa de procesos integrados que asigna las lecturas a un genoma de referencia y encuentra variantes entre el genoma de referencia y las lecturas de la secuencia. Está diseñado con la velocidad en mente, y produce un conjunto coherente de archivos de salida en una sola carpeta.

Snippy sólo requiere tres entradas

1. Genoma de referencia en formato FASTA
2. Archivo(s) de lectura de secuencias en formato FASTQ
3. Una carpeta para  los resultados

El siguiente comando se utiliza para ejecutar Snippy:

In [ ]:
# Ejecutar snippy
!snippy --outdir GPSC46_folder --R1 17150_4#79_1.fastq.gz --R2 17150_4#79_2.fastq.gz --ref Reference_sequence_GPSC46.fa --cpus 2 --ram 2 --force --quiet

La explicación de este comando es la siguiente:

`snippy`: es la herramienta

`--outdir`: especifica el directorio de salida que es GPSC46_folder

`--R1`: especifica la primera lectura que es 17150_4#79_1.fastq.gz

`--R2`: especifica la primera lectura que es 17150_4#79_2.fastq.gz

`--ref`: especifica el genoma de referencia, que es Reference_sequence_GPSC46.fa

`--cpus`: especifica el número máximo de núcleos de CPU a utilizar = 2

`--ram`: especifica que la RAM se mantenga por debajo de 2 GB

`--force`: fuerza la sobreescritura de la carpeta de salida existente

`--quiet`: sin salida en pantalla


Ahora debería haber una nueva carpeta llamada GPSC46_folder. Ahora vamos `cd` a esta carpeta y listamos `ls -alh` el contenido del directorio para comprobar que tenemos nuestros nuevos archivos, y también comprobar sus tamaños. Deberá ver lo siguiente:

A continuación se describen los archivos de salida:

`ref.fa`: Versión/copia FASTA de la referencia

`ref.ra.fai`: Índice del archivo de referencia

`snps.alineado.fa`: Una versión de la referencia pero con (-) en la posición con profundidad=0 y N para 0 < profundidad < --mincov (no tiene variantes)

`snps.bam`: Los alineamientos en formato BAM. Incluye lecturas no mapeadas, multimapping. Excluye duplicados.

`snps.bam.bai`: Índice del archivo .bam

`snps.bed`: Las variantes en formato Browser Extensible Data (BED)

`snps.consensus.fa`: Una versión del genoma de referencia que contiene todas las variantes de alta calidad

`snps.consensus.subs.fa`: Una versión del genoma de referencia que contiene sólo SNPs de alta calidad (no INDELS)

`snps.tab`: Un resumen simple separado por tabuladores de todas las variantes

`snps.csv`: Una versión separada por comas del archivo .tab

`snps.txt`: Lista de columnas separadas por tabulaciones de estadísticas de alineación/tamaño del núcleo

`snps.filt.vcf`: Las llamadas de variantes filtradas de Freebayes

`snps.raw.vcf`: Las llamadas de variantes sin filtrar de Freebayes

`snps.vcf`: Archivo VCF multimuestra con etiquetas GT de genotipo para todos los alelos descubiertos

`snps.vcf.gz`: Archivo .vcf comprimido

`snps.vcf.gz.csi`: Índice para el .vcf.gz a través del índice de bcftools

`snps.gff`: Las variantes en formato GFF3

`snps.html`: Una versión HTML del archivo .tab

`snps.log`: Un archivo de registro con los comandos ejecutados y sus salidas

Snippy ha hecho varias cosas. Puede examinar el fichero de registro para ver exactamente qué ha hecho snippy. Para ello escriba:

In [ ]:
!less snps.log

El resultado es el que se muestra en la tabla siguiente:

También puede utilizar `grep` para recuperar las líneas relevantes para cada comando del archivo de registro.

In [ ]:
!grep outdir snps.log

Obtendrá este resultado:

Para ver los pasos de procesamiento de bwa escriba el comando:

In [ ]:
!grep bwa snps.log

Obtendrá este resultado:

El comando `bwa mem -Y -M -R '@RG\tID:GPSC46_folder\tSM:GPSC46_folder' -t 2 reference/ref.fa /data/17150_4#79_1.fastq.gz /data/17150_4#79_2.fastq.gz | samclip --max 10 --ref reference/ref.fa. fai | samtools sort -n -l 0 -T /tmp --threads 0 -m 1000M | samtools fixmate -m --threads 0 - - | samtools sort -l 0 -T /tmp --threads 0 -m 1000M | samtools markdup -T /tmp --threads 0 -r -s - - > snps.bam` es una combinación de varios comandos combinados mediante pipes `|`. 

La descripción de este comando es la siguiente

`bwa mem -Y -M -R '@RG\tID:GPSC46_folder\tSM:GPSC46_folder' -t 2 reference/ref.fa /data/17150_4#79_1.fastq.gz /data/17150_4#79_2.fastq.gz`: Este comando utiliza bwa para mapear las lecturas de secuencia al genoma de referencia utilizando bwa mem.

`samclip --max 10 --ref referencia/ref.fa.fai`: Este comando utiliza samtools para recortar/eliminar las lecturas no alineadas.

`samtools sort -n -l 0 -T /tmp --threads 0 -m 1000M`: Este comando le dice a samtools que rellene las coordenadas de la pareja e inserte los campos de tamaño.

`samtools fixmate -m --threads 0 - -`: Este comando indica a samtools que rellene las coordenadas de la pareja e inserte los campos de tamaño.

`samtools sort -l 0 -T /tmp --threads 0 -m 1000M`: Este comando le dice a samtools que ordene basándose en el número de cromosoma y las coordenadas

`samtools markdup -T /tmp --threads 0 -r -s - - `: Este comando utiliza samtools para eliminar todos los duplicados y también imprimir algunas estadísticas básicas sobre el archivo de resultados.

`>snps.bam`: snps.bam es el archivo de salida

Para ver el procesamiento de samtools, escriba el comando:

In [ ]:
!grep samtools snps.log

Obtendrá este resultado:

Este comando utiliza freebayes para identificar posibles variantes con coordenadas en formato de archivo de llamada de variantes (VCF )

Para ver el procesamiento de bcftools escriba el comando:

In [ ]:
!grep bcftools snps.log

Obtendrá este resultado:

El comando `bcftools view --include 'FMT/GT="1/1" && QUAL>=100 && FMT/DP>=10 && (FMT/AO)/(FMT/DP)>=0' snps.raw.vcf | vt normalize-r reference/ref. fa - | bcftools annotate --remove '^INFO/TYPE,^INFO/DP,^INFO/RO,^INFO/AO,^INFO/AB,^FORMAT/GT,^FORMAT/DP,^FORMAT/RO,^FORMAT/AO,^FORMAT/QR,^FORMAT/QA,^FORMAT/GL' > snps.filt.vcf ` utiliza bcftools para filtrar las variantes identificadas anteriormente utilizando umbrales fijos.

El comando `bcftools consensus --sample GPSC46_folder -f reference/ref.fa -o snps.consensus.fa snps.vcf.gz` utiliza bcftools para crear un pseudogenoma sustituyendo las bases del genoma de referencia por las sustituciones filtradas (SNPs) identificadas anteriormente.

## Parte 2: Visualizando el resultado

El formato de salida estándar para almacenar la información del llamado de variantes el el VCF. [Aquí](https://samtools.github.io/hts-specs/VCFv4.2.pdf) puede encontrar más información para su interpretación.

Para visualizar el VCF ejecute el comando:

In [ ]:
!head -n30 snps.vcf

Obtendrá el siguiente resultado:

Este comando imprime las 30 primeras líneas del archivo snps.vcf.

Las primeras 27 líneas son "cabeceras" y contienen información sobre lo que se ha hecho para llamar a las variantes, y le ayuda a interpretar lo que significan las diferentes columnas. Todas estas líneas comienzan con "#".
Las 3 últimas líneas son variantes individuales identificadas, una por línea. Las columnas de variantes están etiquetadas como #CHROM POS ID REF ALT QUAL FILTER INFO FORMAT GPSC46_folder.

Podemos ver un resumen algo más sencillo de estas variantes en el archivo snps.tab

In [ ]:
!head -n5 snps.tab

Obtendrá el siguiente resultado:

Este comando imprime las 5 primeras líneas del archivo snps.vcf. En este archivo, no hemos proporcionado información del gen, por lo que sólo son relevantes las 6 primeras columnas.

## Parte 3: Pseudogenomas

Snippy también ha creado un archivo fasta con nuestro pseudogenoma de secuencias que se realiza sustituyendo las bases de referencia por los snps identificados.

In [ ]:
!head snps.consensus.subs.fa

Obtendrá la siguiente salida:

Este comando imprime las primeras líneas del archivo snps.consensus.subs.fa.

## Parte 4: Manejo de múltiples genomas - creación de alineamiento de secuencias múltiples

Para ejecutar un conjunto de secuencias aisladas (reads o contigs) contra la misma referencia, puede utilizar el script snippy-multi.

Navegue hasta la carpeta multiple_files y explore su contenido. Para ejecutar el script snippy-multi, necesitará:

- Lecturas pareadas (lecturas de extremo único o contigs ensamblados)
- Archivo de entrada separado por tabuladores (que contiene una lista de las reads forward y reverse en el siguiente formato: ID, nombres de las lecturas R1 y nombres de las lecturas R2)
- Secuencia de referencia

Explore el archivo de entrada separado por tabulaciones; debe tener el siguiente formato:



Ahora vamos a utilizar snippy-multi para generar un archivo de alineación de secuencias múltiples.

Primero ejecute este comando para generar un script de salida:

In [ ]:
!snippy-multi list.txt --ref Reference_sequence_GPSC46.fa --cpus 2 > runme.sh

La explicación de este comando es la siguiente:

`snippy-multi`: es la herramienta

`list.txt`: Archivo de entrada separado por tabuladores (que contiene una lista de las lecturas directa e inversa en el siguiente formato: ID, nombres de las lecturas R1 y nombres de las lecturas R2)

`--ref`: especifica el genoma de referencia que es Reference_sequence_GPSC46.fa

`--cpus`: especifica el número máximo de núcleos de CPU a utilizar = 2

`>runme.sh`: es el script de salida

Obtendrá la siguiente salida:


Puedes comprobar que el script tiene sentido ejecutando el comando:

In [ ]:
!less runme.sh

Obtendrá este resultado si el script es correcto:

Ahora, genere la alineación ejecutando el comando:

In [ ]:
!sh ./runme.sh

La explicación de este comando es la siguiente:

`sh`: es el nombre del comando del shell Bourne, el intérprete del lenguaje de comandos estándar de Unix y de muchos sistemas operativos similares a Unix, incluido Linux

`/.runme.sh`: el script de salida en el directorio actual `/.`

>Nota: la herramienta snippy-multi también ejecutará snippy-core para generar los archivos de alineación SNP del núcleo del genoma core.*

Snippy ha creado ahora un número de archivos, incluyendo un 'core SNP alignment'.

Ejecute este comando para comprobar el resultado:

In [ ]:
!ls -l core.*

La explicación de estos comandos es la siguiente:

`ls -l` es el comando para listar todos los elementos con core como prefijo (core.*)

Obtendrá la siguiente salida:

Visualización de resultados

Tenemos varios archivos que resumen nuestras variantes, por ejemplo

Ejecute este comando para comprobar el contenido de algunos de estos archivos:

In [ ]:
!head core.tab

Obtendrá la siguiente salida:

Y nuestro alineamiento múltiple de secuencias que contiene todos los genomas está en el archivo (core.tull.aln). Ver este archivo:

In [ ]:
!head core.full.aln

Obtendrá la siguiente salida:

Este archivo "core.full.aln" enmascara secuencias con baja confianza de diferentes maneras, pero para algunas aplicaciones queremos todo enmascarado de la misma manera. Cambiemos eso para que cualquier cosa incierta sea marcada como 'N' usando el script snipp-clean_full_aln que viene con snippy.

In [ ]:
!snippy-clean_full_aln core.full.aln > clean.full.aln

Ver el archivo: 

In [ ]:
!head clean.full.aln

Obtendrá la siguiente salida: